In [1]:
pip install pandas openpyxl lxml



[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: python3.10 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [15]:
import pandas as pd
from lxml import etree
from itertools import islice
from collections import defaultdict
import json

In [3]:
PPC_BOM_PATH = 'ppc_bom.xlsx'
PPC_BOM_SHEET = 'ReportUnitCost20221212'
PPC_CLEANED_BOM_PATH = 'ppc_bom_cleaned.csv'

PCT_BOM_PATH = 'pct_bom_cm_002_v536.xlsx'
PCT_BOM_SHEET = 'data'
PCT_CLEANED_BOM_PATH = 'pct_bom_cleaned.csv'

In [87]:
# Flatten  the JSON Bom aggregate file so we get it in a easier processed format 
json_file = open("bomFgrPosAggregate.json")
bom_aggregate_json = json.load(json_file)
# normalized_bom_aggregate = pd.json_normalize(bom_aggregate_json,meta=['Art', 'Fgrp', 'Pos',['Row','PartNumber'], ['Row', 'Quantity'], ['Row', 'HasCADMatch']], record_path=['Rows','CadParts'] )
normalized_bom_aggregate = pd.json_normalize(bom_aggregate_json,meta=['Art', 'Fgrp', 'Pos', 'VariantRule',['Row','PartNumber'], ['Row', 'Quantity'], ['Row', 'HasCADMatch'],['Row','CADPart', 'Id'], ['Row', 'CADPart', 'CarryOver'], ['Row', 'CADPart', 'KdpMatch']], record_path=['Rows','CadParts', 'Materials'] )

# Save the matched parts to csv
normalized_bom_aggregate.to_csv('bom_aggregate_flatten.csv',index=False)

In [98]:
normalized_bom_aggregate.index.size

12107

In [90]:
normalized_unmatched_cadparts_fgrpos = pd.json_normalize(bom_aggregate_json, meta=['Art', 'Fgrp', 'Pos', 'VariantRule',['CADPart', 'Id'], ['CADPart', 'CarryOver'], ['CADPart', 'KdpMatch']],record_path=['UnmatchedCadParts', 'Materials'] )

In [99]:
normalized_unmatched_cadparts_fgrpos.index.size

14174

In [93]:
normalized_unmatched_cadparts_fgrpos.to_csv("unmatched_cadparts_flatten.csv", index=False)

In [56]:
fgrpos_bom_aggregate = pd.read_json('bomFgrPosAggregate.json')
fgrpos_bom_aggregate

,Art,Fgrp,Pos,Rows,VariantRule,UnmatchedCadParts
0,700,8210,50,"[{'PartNumber': '30622598', 'Quantity': 4.0, '...",V536310VSOPCHPLV,"[{'Id': 'id323029', 'CarryOver': False, 'KdpMa..."
1,700,8250,10,"[{'PartNumber': '30622598', 'Quantity': 12.0, ...",V536310VSOPCHPLV,"[{'Id': 'id326798', 'CarryOver': False, 'KdpMa..."
2,830,2681,50,"[{'PartNumber': '30622624', 'Quantity': 1.0, '...",V536310VSOPCHPLV,"[{'Id': 'id389955', 'CarryOver': False, 'KdpMa..."
3,590,3725,50,"[{'PartNumber': '30622876', 'Quantity': 3.0, '...",V536310VSOPCHPLV,"[{'Id': 'id236235', 'CarryOver': False, 'KdpMa..."
4,640,7612,10,"[{'PartNumber': '30622916', 'Quantity': 4.0, '...",V536310VSOPCHPLV,"[{'Id': 'id262070', 'CarryOver': False, 'KdpMa..."
...,...,...,...,...,...,...
654,750,8341,43,"[{'PartNumber': '32149701', 'Quantity': 1.0, '...",V536310VSOPCHPLV,"[{'Id': 'id353773', 'CarryOver': False, 'KdpMa..."
655,750,8341,44,"[{'PartNumber': '32149701', 'Quantity': 1.0, '...",V536310VSOPCHPLV,"[{'Id': 'id353794', 'CarryOver': False, 'KdpMa..."
656,750,8341,10,"[{'PartNumber': '32149710', 'Quantity': 1.0, '...",V536310VSOPCHPLV,"[{'Id': 'id352975', 'CarryOver': False, 'KdpMa..."
657,720,8970,360,"[{'PartNumber': '32239037', 'Quantity': 1.0, '...",V536310VSOPCHPLV,"[{'Id': 'id375504', 'CarryOver': False, 'KdpMa..."


In [41]:
normalized_bom_aggregate

,Id,CarryOver,KdpMatch,Materials
0,id323150,False,OK,"[{'Name': None, 'Coating': None, 'Material': '..."
1,id323159,False,OK,"[{'Name': None, 'Coating': None, 'Material': '..."
2,id323185,False,OK,"[{'Name': None, 'Coating': None, 'Material': '..."
3,id323190,False,OK,"[{'Name': None, 'Coating': None, 'Material': '..."
4,id323092,False,OK,[]
...,...,...,...,...
7309,id322966,False,OK,[]
7310,id322973,False,OK Inherited,"[{'Name': 'connection', 'Coating': '', 'Materi..."
7311,id322982,False,OK Inherited,"[{'Name': 'PartBody', 'Coating': 'ZM50/50', 'M..."
7312,id268035,False,OK,"[{'Name': 'SENSOR FOIL', 'Coating': '', 'Mater..."


In [19]:
# df = pd.read_json("myJson.json")
fgrpos_bom_aggregate.Rows = pd.DataFrame(fgrpos_bom_aggregate.Rows.values)
# df = df.groupby(['date','name','number'])['locations'].apply(','.join).reset_index()

In [30]:
values = pd.DataFrame(fgrpos_bom_aggregate.Rows[0])
values

,PartNumber,Quantity,HasCADMatch,CadParts
0,30622598,4.0,True,"[{'Id': 'id323150', 'CarryOver': False, 'KdpMa..."
1,30622598,4.0,False,[]
2,32297122,2.0,False,[]
3,32297122,2.0,False,[]
4,32348408,1.0,False,[]
5,32297122,1.0,False,[]
6,31424967,2.0,True,"[{'Id': 'id323092', 'CarryOver': False, 'KdpMa..."
7,32348408,1.0,False,[]
8,32297122,1.0,False,[]
9,31424967,2.0,False,[]


In [16]:
json.load(fgrpos_bom_aggregate[fgrpos_bom_aggregate["Fgrp"] == 8250]["Rows"])

AttributeError: 'Series' object has no attribute 'read'

In [4]:
fgrpos_cad_bom = pd.read_json('fgrPosCadBom.json')
# query Query for PRINS
# {
#   fgPos(where: { number_IN: ["050"] AND: { fgNumber_IN: ["8210"]}}) {
#     id
#     topNode 
#     fgNumber 
#     number
#   	partAddresses {
#       number 
#       name
#       partUsages {id quantity part {partNumber}}
#     }
#   }}

In [125]:
variant_descriptions = pd.read_csv('variant_descriptions.csv', na_values=['""'])
variant_descriptions['VARIANTS'] =  variant_descriptions['VARIANTS'].str.strip()
variant_descriptions.dropna(inplace=True)
# variant_descriptions[variant_descriptions['VARIANTS'] != '']
variant_descriptions.drop(variant_descriptions[variant_descriptions['VARIANTS'] == ''].index, inplace=True)
display(variant_descriptions.dtypes)
variant_descriptions.sort_values(by = "VARIANTS", ascending=True, inplace=True)
display(f'Length: {len(variant_descriptions)}')
variant_descriptions.to_csv('variant_descriptions_cleaned.csv', index=False)
len(variant_descriptions[variant_descriptions['VARIANTS'] == ''])

VARIANTS    object
dtype: object

'Length: 132'

0

In [114]:
plmxml_variant_descriptions = pd.read_csv('plmxml_variants.csv')
display(plmxml_variant_descriptions.dtypes)
# plmxml_variant_descriptions.dropna(inplace=True)
plmxml_variant_descriptions.drop_duplicates(inplace=True)
# plmxml_variant_descriptions['VARIANTS'] =  variant_descriptions['VARIANTS'].str.strip()
plmxml_variant_descriptions.sort_values(by = "VARIANTS", ascending=True, inplace=True)
display(f'Length: {len(plmxml_variant_descriptions)}', plmxml_variant_descriptions)
# plmxml_variant_descriptions.to_csv('plmxml_variants_cleaned.csv', index=False)


VARIANTS    object
dtype: object

'Length: 174'

,VARIANTS
0,-735
1,-MFK 10
3,-USER
2,0
4,0L
...,...
169,TRANSP
170,TRIANGLE
171,TRM13F
172,V536


In [115]:
variants = variant_descriptions['VARIANTS'].to_list()
plm_variants = plmxml_variant_descriptions['VARIANTS'].to_list()
display(plm_variants)

not_matched_variants = variant_descriptions[~variant_descriptions['VARIANTS'].isin(plm_variants)]
not_matched_variants.to_csv("not_matched_variants.csv")


list_of_variants = plmxml_variant_descriptions[plmxml_variant_descriptions.query(f'VARIANTS not in {variants}')]
display(list_of_variants)

['-735',
 '-MFK 10',
 '-USER',
 '0',
 '0L',
 '1234YF',
 '1EDT',
 '265/40 2',
 '295R35 2',
 '5/7 SEAT',
 '7SEAT',
 'ACC PA',
 'ACTSP',
 'ADM HIGH',
 'AIDKIT2',
 'AL90X22D',
 'ALARM',
 'ALOGBASE',
 'AQS',
 'AQUABL',
 'AR10X22D',
 'AUDIO 3',
 'AWD',
 'BELT B',
 'BEV',
 'BLISCTA',
 'BLK DEC2',
 'BUMP BHS',
 'CCD INT',
 'CHARLEST',
 'CHAS ACT',
 'CHATT',
 'COM 110',
 'CORD 2J',
 'COSPAB',
 'COVER',
 'CRPKIT H',
 'CUSHEX',
 'DAB',
 'DABRAD',
 'DBAG B10',
 'DHU 11',
 'DMS',
 'DPOW',
 'DTPMSX',
 'E400V7',
 'ECAS42',
 'ECC 4',
 'EFAD 1',
 'EMB P10',
 'EMB V536',
 'ERAD 1',
 'EU',
 'EU/OS',
 'EVI 2X',
 'FMDM',
 'FX20',
 'GHUD 3',
 'H5 AGM02',
 'HBAT 11',
 'HEATPUMP',
 'HEMBLEM',
 'HLC',
 'HRDGLAS',
 'HVCH',
 'INCHARGE',
 'INT CHAR',
 'INTILL 3',
 'ISOFIX B',
 'KEYLESS4',
 'L DPOW',
 'LCOV S',
 'LCSP',
 'LED HDB',
 'LHD',
 'LHMIR1',
 'LOADNET',
 'LOCK DL',
 'LOCKBOLT',
 'LUMB 4W',
 'MECHORN',
 'METALLIC',
 'MIR4',
 'MIRINN1',
 'MOULCOL2',
 'MSM 3',
 'N ARAB',
 'N AUS',
 'N BRAZ',
 'N CA/MID',
 'N

ValueError: Boolean array expected for the condition, not object

In [5]:
ppc_bom = pd.read_excel(PPC_BOM_PATH, sheet_name=PPC_BOM_SHEET, header = 2, usecols="B:E,H:Z,AK,BI", na_values=['', ' '])
# ppc_bom_small = pd.read_excel(PPC_BOM_PATH, sheet_name=PPC_BOM_SHEET, header = 2, usecols="B:C,H,J,L,M,P,R,T,V:X,AK")

In [6]:
ppc_bom.columns = ppc_bom.columns.str.replace(r'\n', ' ', regex=True)
# ppc_bom.rename(columns={'SYS PARTS/ CAR': 'Quantity', 'PARTNO': 'PartNumber', } )
ppc_bom.head()

,COST CARR,D1,D1 TITLE,Product Stream,SU,SU TITLE,KU,KU TITLE,KU STATUS,RESP DEPT,...,CM NAME,FGRP,FGRP NAME,POS,PARTNO,PARTNO SUFF,PART NAME,DONOR,SYS PARTS/ CAR,Product Module
0,V536X,292250,VEHICLE TOPHAT,VEHICLE TOPHAT,292284,EXTERIOR FRONT AND REAR,307274,Hood and fender systems,DECID,93520,...,P8 US Inscription 7 seats,8210.0,BONNET,50.0,30622598,000,"FLANGE SCREW M6*14*17,2",MBOM,4.0,Exterior Front & Rear
1,V536X,292250,VEHICLE TOPHAT,VEHICLE TOPHAT,292284,EXTERIOR FRONT AND REAR,307274,Hood and fender systems,DECID,93520,...,P8 US Inscription 7 seats,8250.0,FRONT FENDER,10.0,30622598,000,"FLANGE SCREW M6*14*17,2",MBOM,12.0,Exterior Front & Rear
2,MEP2P,292253,ELECTRIC PROPULSION,ELECTRIC PROPULSION,292300,THERMAL MANAGEMENT,302178,Cooling system V536,DECID,97400,...,P8 US Inscription 7 seats,2681.0,NaN,50.0,30622624,000,RETAINER 22,MBOM,1.0,Thermal Exchange & Distribution
3,V536X,292248,SOFTWARE AND ELECTRONICS PLATFORM,SOFTWARE AND ELECTRONICS PLATFORM,292271,CORE SYSTEM COMPONENTS,292340,Core system components,DECID,94200,...,P8 US Inscription 7 seats,3725.0,NaN,50.0,30622876,000,SIX POINT SOCKET SCREW M5,MBOM,3.0,Computing Infrastructure
4,V536X,292251,VEHICLE PLATFORM,VEHICLE PLATFORM,292292,BODY CONTROL AND ROAD CONTACT,280313,Tuning & control,DECID,92520,...,P8 US Inscription 7 seats,7612.0,SHOCK ABSORBER,10.0,30622916,000,SEMS SCREW M8*30*MATHREAD,MBOM,4.0,Body Control


In [ ]:
# display(ppc_bom.dtypes)

for index, dtype in enumerate(ppc_bom.dtypes):
    if pd.api.types.is_object_dtype(dtype):
        col_name = ppc_bom.columns[index]
        print(f"changing {col_name}'s type {dtype} to string")
        ppc_bom[col_name] = ppc_bom[col_name].astype("string") 

ppc_bom.dtypes


: 

In [ ]:
print(f'Number of rows before clean: {ppc_bom.index.size}')
# Remove Zero Quantity values
ppc_bom_cleaned = pd.DataFrame(ppc_bom[ppc_bom['SYS PARTS/ CAR'] != 0.0])
print(f'Number of rows after clean: {ppc_bom_cleaned.index.size}')


: 

In [ ]:
ppc_bom_cleaned["FGRP"] = pd.to_numeric(ppc_bom_cleaned["FGRP"],downcast="integer")
ppc_bom_cleaned["POS"] = pd.to_numeric(ppc_bom_cleaned["POS"],downcast="integer")
ppc_bom_cleaned["ART no"] = pd.to_numeric(ppc_bom_cleaned["ART no"],downcast="integer").astype(pd.Int64Dtype())
ppc_bom_cleaned["SYS PARTS/ CAR"] = pd.to_numeric(ppc_bom_cleaned["SYS PARTS/ CAR"]).astype(pd.Float64Dtype())

ppc_bom_cleaned.head()




: 

In [ ]:
pct_bom_raw = pd.read_excel(
    PCT_BOM_PATH, PCT_BOM_SHEET, header=0, usecols="A:G,R:T,W")
# TODO check with Martina if we can get the Art number instead. Right now we can't merge the arts due to different naming conventions.
arts_df = ppc_bom[['ART name', 'ART no', 'PSS NAME']]
# pct_bom_raw.merge(arts_df, left_on='Product Area', right_on='ART name')


: 

In [ ]:
pct_bom = pct_bom_raw.copy()

: 

In [99]:
#Move the FgrpPos into to separate columns
pct_bom[['Fgrp', 'Pos']] = pct_bom.FgrpPos.str.split(',', expand=True)
pct_bom.drop(columns=['FgrpPos'], inplace=True)
pct_bom

,Program,Control Model,Product Area,Part Number,Part Description,Program Quantity of parts,Updated By,Reason for update,Ref Quantity,Supplier Info,Fgrp,Pos
0,V536,2,ART REMOTE FUNCTIONS,32375075.0,DHU ROYALTY FOR HMI TOOL,1.0,TNETSHAG,MAXBOM first publish,0.0,AEZKD,3931,400
1,V536,2,ART REMOTE FUNCTIONS,NaN,DOUBAN FM,0.0,SYSTEM,Price imported from SI+/epic,1.0,AEAXM,null,null
2,V536,2,ART REMOTE FUNCTIONS,NaN,QINGTING,0.0,SYSTEM,Price imported from SI+/epic,1.0,AEAW8,null,null
3,V536,2,ART ACTIVE SAFETY FUNCTION,30640943.0,"SIX POINT SOCKET SCREW ST4,8*22",2.0,SYSTEM,Price imported from SI+/epic,0.0,AEKIF,3877,050
4,V536,2,ART ACTIVE SAFETY FUNCTION,32286312.0,DRIVER MONITORING CAMERA DMCI LHD,1.0,SYSTEM,Price Imported from SI+/Epic,0.0,AYSPA,3877,050
...,...,...,...,...,...,...,...,...,...,...,...,...
3348,V536,2,ART SEATS AND SAFETY,32273856.0,OUTER SECTION REAR STRUCTRH,1.0,LMENEZES,"VPC77989: + 0,017 USD",0.0,AEIK0,8530,183
3349,V536,2,ART SEATS AND SAFETY,NaN,FLANGE NUT M8*8,0.0,SYSTEM,Price imported from SI+/epic,2.0,AEKIF,null,null
3350,V536,2,ART SEATS AND SAFETY,31109118.0,FLANGE SCREW M5*20 FASTITE,4.0,SYSTEM,Price imported from SI+/epic,0.0,AEKIF,8864,040
3351,V536,2,ART SEATS AND SAFETY,999400.0,PLASTIC NUT T5*11,4.0,SYSTEM,Price imported from SI+/epic,0.0,D3M0A,3656,110


In [102]:
ppc_bom_cleaned.to_csv(PPC_CLEANED_BOM_PATH, index=False, sep=';', decimal=',')
pct_bom.to_csv(PCT_CLEANED_BOM_PATH, index=False,  sep=",")
